Tester et entraîner un modèle de reconnaissance d'écriture
===
<br>

Les dossiers qui constituent l'archive de la correspondance de Constance de Salm réunissent des documents écrits par plusieurs scribes. La différence des écritures représente une difficulté certaine pour la reconnaissance automatique.

Ce *notebook* explique comment procéder à la reconnaissance automatique de l'écriture dans ce contexte particulier impliquant de multiples écritures.

# Classer les images par mains

Inventorier toutes les mains attestées dans une source n'est pas toujours aisé.

L'objectif reste avant tout de **repérer les mains principales** : celles attestées sur le plus grand nombre de pages. Une main qui ne serait attestée que sur une dizaine de pages ne mériterait pas d'être classée, car entraîner un modèle de reconnaissance d'écriture pour un petit nombre de pages est une perte de temps.

Voici un modèle d'arborescence pour le rangement des fichiers et le nommage des dossiers :
```txt
.
└── traitnt-encours
    ├── img-complet
    │   ├── CdS02_Konv019_0011.jpg
    │   ├── CdS02_Konv019_0012.jpg
    │   ├── CdS02_Konv019_0013.jpg
    │   ├── …
    │   ├── CdS02_Konv019_0033.jpg
    │   ├── CdS02_Konv019_0037.jpg
    │   ├── CdS02_Konv019_0039.jpg
    │   └── …
    │
    ├── mainCdS02_Konv019_01
    │   ├── CdS02_Konv019_0011.jpg
    │   ├── CdS02_Konv019_0012.jpg
    │   ├── CdS02_Konv019_0013.jpg
    │   └── …
    └── mainCdS02_Konv019_02
        ├── CdS02_Konv019_0033.jpg
        ├── CdS02_Konv019_0037.jpg
        ├── CdS02_Konv019_0039.jpg
        └── …
```

Il est important qu'aucune image ne soit placée dans deux dossiers de main différents, même si les deux mains en questions sont toutes deux attestées dans l'image. Le script suivant permet de retourner la liste des doublons éventuels :

In [23]:
import os

# On initie un dictionnaire pour la récupération des noms de fichiers et de leur racine
dictFichiers = {}

# On analyse l'arborescence courante
for racine, dossiers, fichiers in os.walk("./traitnt-encours/"):
    # On boucle sur les fichiers
    for fichier in fichiers:
        # On ne sélectionne que les fichiers .jpg et les fichiers rangés dans des dossiers de main
        if fichier[-3:] == "jpg" and racine[:22] == "./traitnt-encours/main":
            # Si le fichier est absent du dictionnaire, on l'ajoute
            if not dictFichiers.get(fichier):
                dictFichiers[fichier] = [racine]
            # Si le fichier est présent dans le dictionnaire, on ajoute sa racine à la liste des valeurs
            else:
                dictFichiers[fichier].append(racine)
    
# On initie un booléen pour confirmer l'absence de doulon
doublons = False

# Pour rechercher les doublons, on boucle sur les entrées du dictionnaire
for fichier in dictFichiers:
    # Si le nom du fichier est associé à une liste de plus de 1 racine
    if len(dictFichiers[fichier]) > 1:
        doublons = True
        # On retourne alors un message d'alerte
        print(f"Le fichier {fichier} est doublonné :")
        for racine in dictFichiers[fichier]:
            print(f"{racine}/{fichier}")

# On retourne un message si aucun doublon n'a été trouvé
if not doublons:
    print("Bravo ! Aucun doublon n'a été trouvé.")

Bravo ! Aucun doublon n'a été trouvé.


S'il existe des doublons, il est important de **les supprimer**. On peut le faire automatiquement grâce à la commande suivante, qui va supprimer arbitrairement la première occurrence :

In [20]:
# On boucle sur les entrées du dictionnaire
for fichier in dictFichiers:
    # Si le nom du fichier est associé à une liste de plus de 1 racine
    if len(dictFichiers[fichier]) > 1:
        # On supprime le premier de la liste
        !rm {dictFichiers[fichier][0]}/{fichier}

# Créer un échantillon-test de chaque écriture

Afin de pouvoir tester un ou plusieurs modèles, il est nécessaire de constituer une vérité de terrain de 2-3 doubles pages (selon la densité d'écriture qu'elles contiennent).

## Critères de sélection

On crée dans le dossier de chaque main un dossier **test/** contenant des spécimens d'écriture selon les critères suivants :
- Reproductions de bonne qualité (sans problème de transparence)
- Pages choisies de manière discontinue (l'écriture d'une même main peut en effet varier et il est utile de prendre en compte cette variété pour le test)

Si certaines mains ne sont attestées qu'en compagnie d'autres écritures, on veillera à limiter le test de reconnaissance d'écriture aux seuls lignes de la main à tester (en supprimant après segmentation les lignes non pertinentes afin que la reconnaissance de l'écriture ne les traite pas).

## Organisation des fichiers

Voici un modèle d'arborescence pour le rangement et le nommage des fichiers et dossiers. Il est important de **déplacer les fichiers** vers le dossier test/ et non **pas les copier** :
```txt
.
└── traitnt-encours
    ├── img-complet
    │   ├── CdS02_Konv019_0011.jpg
    │   ├── CdS02_Konv019_0012.jpg
    │   ├── CdS02_Konv019_0013.jpg
    │   ├── …
    │   ├── CdS02_Konv019_0033.jpg
    │   ├── CdS02_Konv019_0037.jpg
    │   ├── CdS02_Konv019_0039.jpg
    │   └── …
    │
    ├── mainCdS02_Konv019_01
    │   ├── CdS02_Konv019_0013.jpg
    │   ├── …
    │   ├── test
    │   │   ├── CdS02_Konv019_0011.jpg
    │   │   └── CdS02_Konv019_0012.jpg
    │   └── train
    │
    └── mainCdS02_Konv019_02
        ├── CdS02_Konv019_0039.jpg
        ├── …
        ├── test
        │   ├── CdS02_Konv019_0033.jpg
        │   └── CdS02_Konv019_0037.jpg
        └── train
```

## Démarche

1. On suit de manière rigoureuse les préconisations pour la transcription énoncées dans la [documentation analytique](https://github.com/sbiay/CdS-edition/blob/main/documentation/documentation.pdf) (cf. annexe Normes de transcription).
2. On procède à la segmentation automatique puis à l'annotation manuelle des pages (comme expliqué dans le notebook [suivant](./3_Segmenter_et_annoter_une_page.ipynb)) en éliminant manuellement toutes les lignes d'une écriture différente de celle pour laquelle la collection d'entraînement est faite ;
3. On transcrit les pages ;
4. On exporte les pages d'entraînement au format XML-Alto et on les place dans le dossier **test** de la main concernée.


# Tester des modèles HTR

## Installer l'application Kraken

On recommande pour tester un modèle d'utiliser l'application Kraken en ligne de commande, disponible pour Linux et Mac OSX (non pour Windows). Les instructions sont consultables [ici](https://github.com/mittagessen/kraken#installation).

## Importer un modèle

Les modèles extérieurs au projet que l'on a utilisés sont téléchargeables sur le [Gitlab du laboratoire Inria](https://gitlab.inria.fr/dh-projects/kraken-models/-/tree/master/transcription%20models) :
- generic_lectaurep_26.mlmodel
- cm_ft_mrs15_11.mlmodel

## Initier un journal de tests

Le script [journalReconn.py](./py/journalReconn.py) permet de pré-remplir un journal pour l'enregistrement des résultats des tests effectués sur les modèles. 

On lui donne comme argument un nom de modèle et il écrit dans le fichier Json [journal-rec.json](./sources/journal-rec.json), à la date et à l'heure courante et pour chaque main listée dans le fichier [mains.csv](./sources/mains.csv), les noms de ces mains et prépare le renseignement des valeurs de test.

Conseils d'utilisation :
- Paramètre MODELE : obligatoire ; reseigner un **nom de modèle** plutôt que son chemin relatif ou absolu
- Option -v, --veriteterrain : le test initial se fait naturellement sans avoir entraîné le modèle sur les vérités de terrain
- Option -i, --ignore : pour ignorer une main listée dans le fichier [mains.csv](./sources/mains.csv)

In [ ]:
!python3 py/journalReconn.py MODELE

Le contenu écrit dans le fichier journal donne comme **accuracy** pour chaque main une valeur de 0. Cette valeur doit être saisie manuellement dans le fichier une fois effectué le test comme suit.


## Effectuer un test

Avec la commande suivante on effectue un test d'acuité pour un modèle sur une main particulière (on doit relever l'*Average accuracy*) :

In [ ]:
# ketos test -m ../sources/modeles-rec/NOM-MODEL.mlmodel sources/NOM-MAIN/test/*xml -f alto
!ketos test -m ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel sources/mainCdS02_Konv019_02/test/*xml -f alto

Il convient de choisir le modèle présentant **les meilleurs résultats pour l'ensemble des écritures testées**. Si le meilleur modèle n'atteint pas 90% pour tout ou partie des mains, il convient de l'entraîner par l'apport de vérités de terrain pour chaque main n'atteignant pas ce score.

# Entraîner un modèle

## Principe général

L'entraînement d'un modèle multi-main consiste à constituer une collection d'entraînement pour chaque main concernée et à entraîner un modèle sur l'ensemble de ces collections réunies.

## Constituer des collections d'entraînement

On procède à la constitution d'une vérité de terrain pour chaque spécimen d'écriture. Il s'agit de transcrire l'**équivalent d'une dizaine de pages simples** : si les pages présentent des blancs importants ou d'autres écritures étrangères à l'entraînement (elles ne doivent pas être transcrites) on augmentera le nombre de pages pour parvenir grosso modo à ce volume de 10 pages simples.

La transcription de ces pages se fait à la main, sans l'aide d'une première reconnaissance de l'écriture, car il est plus compliqué de corriger une mauvaise prédiction que de transcrire manuellement. On reprend pour cela la [démarche](./2_Tester_et_entrainer_un_modele_HTR.ipynb#D%C3%A9marche) énoncée plus haut.

## Entraîner un modèle

Une fois les collections d'entraînement de chaque main, on écrit dans un fichier dédié les chemins de fichiers des vérités de terrain sur lesquelles l'entraînement doit s'appuyer (i.e. tous les fichiers XML contenus dans les dossiers **train/**) :

In [ ]:
# On écrit la liste des chemins de fichiers
!find -wholename */train/*xml > train.txt

# On crée un dossier de sortie pour le modèle entraîné
!mkdir --parents ./sources/modeles-rec/out/

# La commande suivante entraîne le modèle cm_ft_mrs15_11.mlmodel
!ketos train -r 0.0001 --lag 20  -s '[1,120,0,1 Cr3,13,32 Do0.1,2 Mp2,2 Cr3,13,32 Do0.1,2 Mp2,2 Cr3,9,64 Do0.1,2 Mp2,2 Cr3,9,64 Do0.1,2 S1(1x0)1,3 Lbx200 Do0.1,2 Lbx200 Do.1,2 Lbx200 Do]' --augment --device cpu --resize add -i ./sources/modeles-rec/cm_ft_mrs15_11.mlmodel -t ./sources/train.txt -f alto --output --output sources/modeles-rec/out/

## Tester le nouveau modèle

Une fois le modèle entraîné, on renouvelle la procédure de test expliquée [ci-dessus](./2_Tester_et_entrainer_un_modele_HTR.ipynb#Tester-des-mod%C3%A8les-HTR). Si certaines mains ont toujours un score inférieur à 90%, on réitère la procédure d'entraînement en apportant 5 nouvelles pages pour chaque main n'atteignant pas le score souhaité.